## Scrape_tweets_script_filter

- [NLTK documentation](https://www.nltk.org/howto/collocations.html)
- [sentiment analysis tutorial](https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk)
- [Scrape tweets from twitter](https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1)

This script performs the folling functions 

- Import necessary libraries 
- Script1 - Scrape tweets from homepage 
- Script 2 - Class method - Scrape tweets from a particular user - **need screen_name** include retweets
- Script 3 - Class method - Scrape tweets from a particular user - **need screen_name** Exclude retweets 
- Script 4 - Search tweets with keyword and date
- Script 5 - Profanity filter 
- Script 6-  Profanity Censor - Script will censor words but leave tweets intact
- Script 7 -  Censor tweets in the dataframe 


In [1]:
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords') #Dont need this unless stopwords need to be updated
import string
#This package will allow us to remove punctuation
from nltk.tokenize import RegexpTokenizer
#import regex
#import twitter scraper 
import tweepy
#Import Json to allow us to get an organize the info from twitter
import json
#Import datetime to use in our dataframe 
import datetime
#import pandas to create and manipulate dataframes 
import pandas as pd
#import profanity checker 
from profanity_check import predict, predict_prob
#Import profanity Filter 
from profanity_filter import ProfanityFilter

#### Twitter API keys
- consumer_key = "****"
- consumer_secret = "****"
- access_token = "****"
- access_token_secret = "****"


### Script 1 -  Scrape tweets from homepage

In [4]:
#This script will print tweets from my homepage - it is unstructured

#Add your credentials here
twitter_keys = {
        'consumer_key':        '****',
        'consumer_secret':     '****',
        'access_token_key':    '****',
        'access_token_secret': '****'
    }

#Setup access to API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth)

#Make call on home timeline, print each tweets text
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

Opinion: Beaten up and down a third of their original squad, the Wallabies need a hero to take the third Test by th… https://t.co/Qpmcs826u8
Planning to fight Russia and being prepared to do so are two different things https://t.co/mx4gPC0bdl
I like the sound of this https://t.co/o3u1H8KHVh
Non-white ICU patients get less oxygen treatment than needed -study https://t.co/4UCKjdCoBA https://t.co/3tfoKwGkM3
Away with the lot of them ! https://t.co/ve3GNBaxBa
Do you know the photographer on this image? https://t.co/UBvnq5jMLd
Morning mail: Uber whistleblower revealed, Albanese honeymoon over, drama on the Spanish Steps https://t.co/A8fVYbviHm
President Biden is set to unveil a “deep field” image captured by the James Webb Space Telescope — the largest spac… https://t.co/onTvDQ6A2Q
Three Arrows liquidators asks U.S. court to force crypto founders to cooperate https://t.co/lvAzZ8r1St https://t.co/4krv1V6nZU
It's happy hour somewhere. https://t.co/5l7vzoQAnl
wars for my money. I hope to see R

In [29]:
# Print all of the metadata from the above tweets
#public_tweets[0]

In [6]:
# Organize metadata info to determine what to use 
status = public_tweets[0]

#convert to string
json_str = json.dumps(status._json)

#deserialise string into python object
parsed = json.loads(json_str)

print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Mon Jul 11 21:16:44 +0000 2022",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [
            {
                "display_url": "twitter.com/i/web/status/1\u2026",
                "expanded_url": "https://twitter.com/i/web/status/1546604428186230789",
                "indices": [
                    117,
                    140
                ],
                "url": "https://t.co/Qpmcs826u8"
            }
        ],
        "user_mentions": []
    },
    "favorite_count": 1,
    "favorited": false,
    "geo": null,
    "id": 1546604428186230789,
    "id_str": "1546604428186230789",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "is_quote_status": false,
    "lang": "en",
    "place": null,
    "possibly_sensitive": false,
    "possibly_sensitiv

### Script 2 - Class method
##### Scrape tweets from a particular user - **need screen_name** include retweets

In [7]:
#include retweets

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        '****',
        'consumer_secret':     '****',
        'access_token_key':    '****',
        'access_token_secret': '****'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="AOC", #Alexandra Ocasio Cortez 
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

### Script 3 - Class method
##### Scrape tweets from a particular user - **need screen_name** Exclude retweets 

In [8]:
#Remove RETWEETS 
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':        '****',
        'consumer_secret':     '****',
        'access_token_key':    '****',
        'access_token_secret': '****'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="MMFlint", #Michael Moore 
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended'
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended')
                
            for item in statuses:

                mined = {
                    'tweet_id':        item.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'retweet_count':   item.retweet_count,
                    'text':            item.full_text,
                    'mined_at':        datetime.datetime.now(),
                    'created_at':      item.created_at,
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'status_count':    item.user.statuses_count,
                    'location':        item.place,
                    'source_device':   item.source
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

In [9]:
miner=TweetMiner(result_limit = 20 )

In [10]:
#Print Mined tweets as a dataframe
mined_tweets = miner.mine_user_tweets(user='MMFlint', max_pages=17)

mined_tweets_df= pd.DataFrame(mined_tweets)

In [11]:
#Print the 7th row to see if script is accurate 
mined_tweets_df.loc[[6]]

,tweet_id,name,screen_name,retweet_count,text,mined_at,created_at,favourite_count,hashtags,status_count,location,source_device,retweet_text,quote_text,quote_screen_name
6,1542208227533791234,Michael Moore,MMFlint,78,RT @estern839: @JillWineBanks https://t.co/5Pk...,2022-07-11 17:17:17.146596,2022-06-29 18:07:48+00:00,0,[],16009,None,Twitter for iPhone,@JillWineBanks https://t.co/5PkUpWMjSN,None,None


### Script 4 - Search tweets with keyword and date
https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/

In [22]:
#Key word search Query is not case sensitive 

search_query = "biden -filter:retweets"

In [25]:
# get tweets from the API
tweets = tweepy.Cursor(api.search_tweets,
              q=search_query,
              lang="en",
              since="2022-07-01").items(10000)
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

In [14]:
#Print scraped tweets to a dataframe 
# intialize the dataframe
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name,
                                               'screen_name' : tweet.user.screen_name,
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)
# show the dataframe
tweets_df.head()

,user_name,screen_name,user_location,user_description,user_verified,date,text,hashtags,source
0,88VIBZ,LShibe88,,"A no~body ! here, to re💎erberate the ech🌒es of...",False,2022-07-11 21:17:28+00:00,@scott4pendleton @TomFitton Trump &amp; Biden ...,None,Twitter for iPhone
1,ULTRA MEGA MAGA 2024,JimMorr87833987,Mar - a - Lago #45 & #47,Main Force Patrol \n\n ...,False,2022-07-11 21:17:28+00:00,@JoeBiden\n@DHSgov\n@Comey\n@FBI\n@TheJusticeD...,None,Twitter for Android
2,Liam,e_Liam_,Earth,"ENTP-A metaphysical realist\n\n~ Truth is one,...",False,2022-07-11 21:17:27+00:00,@EssexPR Does Hunter Biden's Son have videos o...,None,Twitter for Android
3,Dr. Stanch 24/7,Stanch247,"Solana Beach, CA",No quarter asked. None given. Followed by @Ric...,False,2022-07-11 21:17:27+00:00,"@JesseKellyDC Ah, yes. The sniff-grope two-fer...",None,Twitter for iPhone
4,purecarp,purecarp,,Matt 6:10 #MAGA #Trump #1A #2A ; No to sociali...,False,2022-07-11 21:17:26+00:00,One thing that Pedo Peter Joe Refuse to unders...,None,Twitter for Android


In [28]:
#tweets_df.to_csv('biden_tweets_07/11/2022.csv')

tweets_df.to_csv('biden_tweets.csv')


In [26]:
tweets_df.tail()

,user_name,screen_name,user_location,user_description,user_verified,date,text,hashtags,source
2603,Tweetnylawyers,TweetNYLawyers,New York,Tweets by New York Lawyers Success blog. NY & ...,False,2022-07-11 21:04:19+00:00,@DaddyJim52 Then why would you support Biden a...,None,Twitter Web App
2604,Michael McNamara 🇺🇦 🇺🇸 ☮️,Mikebespeckled,,Political science major and political nerd. Pa...,False,2022-07-11 21:04:19+00:00,@DaddyJim52 Then why would you support Biden a...,None,Twitter for iPhone
2605,Eddie_LeBec🟠,Eddie__Lebec,"Deals Gap, AL",,False,2022-07-11 21:04:19+00:00,@DaddyJim52 Then why would you support Biden a...,None,Twitter Web App
2606,B,MosmanBarb,"Sydney, New South Wales",‘Your racism is not my problem.‘ Sandra Bulloc...,False,2022-07-11 21:04:19+00:00,@DaddyJim52 Then why would you support Biden a...,None,Twitter for iPad
2607,Christopher Morvant,Snakebite8974,"Morgan City, LA",If you support the J6C sham “fact finding” Com...,False,2022-07-11 21:04:18+00:00,@DaddyJim52 Then why would you support Biden a...,None,Twitter for iPhone


### Script 5 - Profanity filter 

In [13]:
#This function adds a new column into the dataframe above and uses a boolean value to let you know if it contains profanities    

def detect_profanity(df): 
    df['contains_profanity'] = predict(df['text'])
    return df 


In [14]:
# Run the above function through the dataframe
tweets_df_prof= detect_profanity(tweets_df)
tweets_df_prof

,user_name,screen_name,user_location,user_description,user_verified,date,text,hashtags,source,contains_profanity
0,garfield dingess,DingessGarfield,,,False,2022-07-08 02:35:27+00:00,@CalltoActivism They already do. Biden forget...,None,Twitter Web App,0
1,Rod,dudewtfnow,"Newport Beach, California",🏄 NBA and SkateBoarding 🤙. ...,False,2022-07-08 02:35:26+00:00,Tucker Carlson reverses position on immigratio...,None,Twitter for iPhone,0
2,Ronnie 101,rondonzo101,"Tucson, AZ",Keep America free,False,2022-07-08 02:35:26+00:00,Isn't it amazing that hunter Biden can't speak...,None,Twitter for Android,0
3,Diane Hayes,lemontarting,"Austin, TX",Former atty current mom Voting rights matter. ...,False,2022-07-08 02:35:26+00:00,@DeanObeidallah Remember when Hunter Biden got...,None,Twitter for iPhone,0
4,Air in Gear,eringhere,Minnesota,Pro-abortion. Pro-LGBTQIA+ rights. Working to ...,False,2022-07-08 02:35:26+00:00,Michigan is ripe for support. MI-03 has a GOP ...,None,Twitter Web App,0
...,...,...,...,...,...,...,...,...,...,...
95,Jennifer Girod,JenniferGirod,,,False,2022-07-08 02:34:27+00:00,@Barbara35980709 @bluebear5922 @RepMTG @GOPLea...,None,Twitter for Android,0
96,MyFirstSocialMediaAccount 🇨🇦 🇺🇦,MyFirstSocialM2,,,False,2022-07-08 02:34:27+00:00,@CarolynGarman5 @MacFarlaneNews He agrees Bide...,None,Twitter Web App,0
97,Angela Stadalsky,8NO4RE,,"Wife,mother, patriot, proud Conservative",False,2022-07-08 02:34:27+00:00,If Biden and the Democrats\n Think that they ...,None,Twitter for Android,0
98,Bas Klootwijk,BasKlootwijk1,Salty Spray,Ik ben geïnteresseerd in troep en trollen.\nIk...,False,2022-07-08 02:34:27+00:00,Timcast IRL - #DutchCops FIRE ON #DutchFarmers...,"[DutchCops, DutchFarmers, GlobalAgenda]",Twitter Web App,0


In [15]:
#Print only profanity tweets to see if this script works 
profanity_tweet = tweets_df_prof[tweets_df_prof['contains_profanity'] == 1]
profanity_tweet

,user_name,screen_name,user_location,user_description,user_verified,date,text,hashtags,source,contains_profanity
8,Gregoryr,Dos54,Ca,Retired worked in the auto industry,False,2022-07-08 02:35:22+00:00,Joe Biden should be put in jail for destroying...,None,Twitter Web App,1
35,Mark Bedinger,MarkBedinger,,,False,2022-07-08 02:35:09+00:00,@David_Charts @BidensWins Awww Biden ain’t got...,None,Twitter for iPhone,1
42,Edith,Edith83120,,,False,2022-07-08 02:35:07+00:00,@RyanShead Hunter Biden is trending because hi...,None,Twitter for Android,1
47,Duckfootball1991,Duckfootball192,,.,False,2022-07-08 02:35:03+00:00,@EricOwnby1 @BryanRo89512832 @ClayTravis @NPR ...,None,Twitter for iPad,1


### Script 6-  Profanity Censor

##### Script will censor words but leave tweets intact

https://lubna2004.medium.com/profanity-to-be-or-not-to-be-dd32d53648f7

In [16]:
#Test1
predict(['CRAP! What a stupid decision!'])

#Crap and Stupid are examples of profanities 
#This is delineated by the boolean value of one 

array([1])

In [17]:
#Test2
predict(['Lovely Spring Weather!'])
#There are no profanities contained in this

array([0])

In [18]:
#Test 3 
#Use profanity Filter - to change and censor profane words 

In [19]:
#Initialize profanity filter and what characters will replace profane language
pf = ProfanityFilter()
pf.censor_char = '@'
pf.censor("Crap! What a stupid decision")

'@@@@! What a stupid decision'

In [20]:
#Copy variable
censor_profanity_tweet = profanity_tweet


In [21]:
#Check datatypes - profanity filter can only be run on strings
profanity_tweet['text'].dtypes

dtype('O')

In [22]:
profanity_tweet.dtypes

user_name                          object
screen_name                        object
user_location                      object
user_description                   object
user_verified                        bool
date                  datetime64[ns, UTC]
text                               object
hashtags                           object
source                             object
contains_profanity                  int64
dtype: object

In [23]:
#Convert text column to string
string = profanity_tweet.astype({'text':'string'})


In [24]:
string['text']

8     Joe Biden should be put in jail for destroying...
35    @David_Charts @BidensWins Awww Biden ain’t got...
42    @RyanShead Hunter Biden is trending because hi...
47    @EricOwnby1 @BryanRo89512832 @ClayTravis @NPR ...
Name: text, dtype: string

In [25]:
#Check that 'Text' column has been converted to string
string.dtypes

user_name                          object
screen_name                        object
user_location                      object
user_description                   object
user_verified                        bool
date                  datetime64[ns, UTC]
text                               string
hashtags                           object
source                             object
contains_profanity                  int64
dtype: object

In [26]:
#Copy variable 
string2 = string[["text"]]
string2.dtypes

text    string
dtype: object

In [27]:
#This will allow us to print the entire string for the next cell
pd.set_option('display.max_colwidth', -1)

<ipython-input-27-2c1c50702082>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [28]:
mystring3 = repr(string2)
#Check that profanity censor works 
pf.censor(mystring3)

"                                                                                                                                                                                                                                                              text\n8   Joe Biden should be put in jail for destroying the USA. Can't afford the gas to go to my son's wedding                                                                                                                                                        \n35  @David_Charts @BidensWins Awww Biden ain’t got @@@@ on the Big Dipper. @@@ wipes                                                                                                                                                                              \n42  @RyanShead Hunter Biden is trending because his old man sold man sold barrels of oil to China today from our strategic reserve. Who cares about Trump - the former president - and the former 1st family. We're thinkin

In [29]:
#Compare censored version above to original dataframe
display(mystring3)

"                                                                                                                                                                                                                                                              text\n8   Joe Biden should be put in jail for destroying the USA. Can't afford the gas to go to my son's wedding                                                                                                                                                        \n35  @David_Charts @BidensWins Awww Biden ain’t got shit on the Big Dipper. Ass wipes                                                                                                                                                                              \n42  @RyanShead Hunter Biden is trending because his old man sold man sold barrels of oil to China today from our strategic reserve. Who cares about Trump - the former president - and the former 1st family. We're thinkin

In [43]:
string3 = string

In [45]:
#Initialize censor function to run through the 'text' column - check that function works on tweets containing profanities
def censortweet(x):
   return pf.censor(x)
string3["text"] = string3["text"].apply(censortweet)

In [46]:
string3

,user_name,screen_name,user_location,user_description,user_verified,date,text,hashtags,source,contains_profanity
8,Gregoryr,Dos54,Ca,Retired worked in the auto industry,False,2022-07-08 02:35:22+00:00,Joe Biden should be put in jail for destroying the USA. Can't afford the gas to go to my son's wedding,None,Twitter Web App,1
35,Mark Bedinger,MarkBedinger,,,False,2022-07-08 02:35:09+00:00,@David_Charts @BidensWins Awww Biden ain’t got @@@@ on the Big Dipper. @@@ wipes,None,Twitter for iPhone,1
42,Edith,Edith83120,,,False,2022-07-08 02:35:07+00:00,@RyanShead Hunter Biden is trending because his old man sold man sold barrels of oil to China today from our strategic reserve. Who cares about Trump - the former president - and the former 1st family. We're thinking today. Pull your head out of your @@@,None,Twitter for Android,1
47,Duckfootball1991,Duckfootball192,,.,False,2022-07-08 02:35:03+00:00,@EricOwnby1 @BryanRo89512832 @ClayTravis @NPR Everyone knew what was coming and Biden was threatening them with sanctions if they invaded. I was shocked she would even go there with the existing climate. Dumb dumb move by her and her advisors.,None,Twitter for iPad,1


## Script 7 -  Censor tweets in the dataframe 


In [33]:
# Copy the original 
tweets_df_censor = tweets_df

In [34]:
# Text column to string will allow us to censor 
tweets_df_censor = tweets_df_censor.astype({'text':'string'})
tweets_df_censor.head()

,user_name,screen_name,user_location,user_description,user_verified,date,text,hashtags,source,contains_profanity
0,garfield dingess,DingessGarfield,,,False,2022-07-08 02:35:27+00:00,@CalltoActivism They already do. Biden forget to tell Americans that. We are paying that stimulus money back right now through inflation. Vote red!,None,Twitter Web App,0
1,Rod,dudewtfnow,"Newport Beach, California",🏄 NBA and SkateBoarding 🤙. Some Parody Tweets!,False,2022-07-08 02:35:26+00:00,Tucker Carlson reverses position on immigration after interviewing Hot Blonde Dutch Reporter. He is urging Biden to reopen our borders 🤣 https://t.co/BxQm6MwcIE,None,Twitter for iPhone,0
2,Ronnie 101,rondonzo101,"Tucson, AZ",Keep America free,False,2022-07-08 02:35:26+00:00,Isn't it amazing that hunter Biden can't speak Ukraine Chinese or Russian In all these countries that he owns And has partnership Not to mention any experience or Knowledge of those company,None,Twitter for Android,0
3,Diane Hayes,lemontarting,"Austin, TX",Former atty current mom Voting rights matter. BLM. Democracy is on life support.,False,2022-07-08 02:35:26+00:00,"@DeanObeidallah Remember when Hunter Biden got $2B from the Saudis after helping them murder Jamal Khashoggi? Oh right, that was Trump’s son in law who worked in the White House.",None,Twitter for iPhone,0
4,Air in Gear,eringhere,Minnesota,Pro-abortion. Pro-LGBTQIA+ rights. Working to be anti-racist every day. VOTE. VOTE. VOTE. Vote like you're saving democracy; vote like you're saving our rights.,False,2022-07-08 02:35:26+00:00,"Michigan is ripe for support. MI-03 has a GOP incumbent but was redistricted into a Biden +9 district. Currently a toss up, with recent polls leaning slightly toward the Dems. Big pick up opportunity to offset other losses. 12/?",None,Twitter Web App,0


In [35]:
#Run dataframe through censortweet function from above 
tweets_df_censor["text"] = tweets_df_censor["text"].apply(censortweet)

In [37]:
#Check that function worked - row 14 has the profanities censored using the functions
tweets_df_censor.loc[[8]]




,user_name,screen_name,user_location,user_description,user_verified,date,text,hashtags,source,contains_profanity
8,Gregoryr,Dos54,Ca,Retired worked in the auto industry,False,2022-07-08 02:35:22+00:00,Joe Biden should be put in jail for destroying the USA. Can't afford the gas to go to my son's wedding,None,Twitter Web App,1
